<a target="_blank" href="https://colab.research.google.com/github/cerr/pyCERR-Notebooks/blob/main/load_visualize_scan_seg_ex1.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Loading and visualizig DICOM

This notebook demonstrates importing images in DICOM format to CERR's planC data structure and visualizing the imported objects. Phantom dataset made available by Lambin et al via CancerData and re-distributed along with the CERR package is used in this demonstration. Each patient daatset consists of CT scan for Lung and tumor segmentation.

#### Install pyCERR (development version)

In [ ]:
%%capture
! pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR"

#### Get a list of paths to patient datasets

In [ ]:
from cerr import datasets
import os
phantom_dir = os.path.join(os.path.dirname(datasets.__file__),'radiomics_phantom_dicom')
pat_names = ['pat_1', 'pat_2', 'pat_3', 'pat_4']
all_pat_dirs = [os.path.join(phantom_dir, pat) for pat in pat_names]

#### Import contents of a patient directory to planC

In [ ]:
# Import dicom to planC
from cerr import plan_container as pc
dcm_dir = all_pat_dirs[0]
planC = pc.loadDcmDir(dcm_dir)

#### Metadata access

In [ ]:
print("Number of Scans in planC:")
print(len(planC.scan))
print("Scan types:")
print([s.scanInfo[0].imageType for s in planC.scan])
print("Number of Structures in planC:")
print(len(planC.structure))
print("Structure Names")
print([s.structureName for s in planC.structure])

# Access segmented mask
import cerr.contour.rasterseg as rs
structNum = 0
mask3M = rs.getStrMask(structNum, planC)
print("Number of labeled voxels in the mask")
mask3M.sum()

# Access scan
scanNum = 0
scan3M = planC.scan[scanNum].getScanArray()
print("Size of scanArray")
print(scan3M.shape)

# Compute ROI statistics
print("Mean intensity in ROI:")
print(scan3M[mask3M].mean())

#### Visualize scan and lung-tumor segmentation

Napari package is used for visualization. Napari requires a Display for rendering graphics and does not support visualization on a remote server (e.g. Google Colab)

In [ ]:
from cerr import viewer as vwr
scanNumV = [0]
doseNumV = []
strNumV = [0]
viewer, scan_layer, struct_layer, dose_layer, dvf_layer = \
    vwr.showNapari(planC, scan_nums=scanNumV, struct_nums=strNumV, dose_nums=doseNumV, vectors_dict={}, displayMode='2d')


#### Adjust Visualization settings

Properties of Napari viewer and image "layers" can be readily accessed and updated. Below cell shows an example of changing opacity, contrast ranges and gamma scale.

In [ ]:
scan_layer[0].opacity = 0.5
scan_layer[0].contrast_limits_range = [-1000, 300]
scan_layer[0].contrast_limits = [-900, 200]
scan_layer[0].gamma = 0.7
#dose_layer[0].visible = True

#### Programmatically capturing screenshots

Metadata can be accessed from planC to update the Viewer to programmatically capture image slices and objects of interest. The example below shows screenshot capture of tumor segmentation on central CT slice

In [ ]:
from skimage.io import imsave, imshow
import os
import cerr.contour.rasterseg as rs
import numpy as np
save_dir = r'L:\Aditya\batch_images'
# Get index of central axial slice
scanNum = 0 # index of scan in planC.scan
strNum = 0 # index of structure in planC.structure
mask3M = rs.getStrMask(strNum, planC)
rV, cV, sV = np.where(mask3M)
midSliceInd = int(np.round(sV.mean()))
# update viewer to display the central slice and capture screenshot
xV, yV, zV = planC.scan[scanNum].getScanXYZVals()
viewer.dims.set_point(2, zV[midSliceInd])
screenshot = viewer.screenshot()
capture_fname = os.path.join(save_dir,pat_names[0] + '.png')
imsave(capture_fname, screenshot)
print('saved to '+ capture_fname)

#### Show the saved image

In [ ]:
im = imshow(capture_fname)
im.axes.set_axis_off()

#### DVH-based features

In [ ]:
from cerr import dvh
structNum = 5
doseNum = 0
binWidth = 0.05
percent = 70
dosesV, volsV, isErr = dvh.getDVH(structNum, doseNum, planC)
doseBinsV,volHistV = dvh.doseHist(dosesV, volsV, binWidth)
dvh.MOHx(doseBinsV,volHistV,percent)